In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#ignore warning
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
!pip install notexbook-theme
%reload_ext notexbook
%texify

**We will use the pre-trained Keras FaceNet model provided by Hiroki Taniai in this tutorial. It was trained on MS-Celeb-1M dataset and expects input images to be color, to have their pixel values whitened (standardized across all three channels), and to have a square shape of 160×160 pixels.**

# **1. DETECT FACES**

**Face detection** is the process of automatically locating faces in a photograph and localizing them by drawing a bounding box around their extent.

we will use **the Multi-Task Cascaded Convolutional Neural Network**, or MTCNN, for face detection, e.g. finding and extracting faces from photos. This is a state-of-the-art deep learning model for face detection .

Multi-task Cascaded Convolutional Networks (MTCNN) is a framework developed as a solution for both face detection and face alignment.

**install mtcnn**

In [4]:
!pip install mtcnn

confirm that the library was installed correctly by importing the library and printing the version.

In [5]:
import mtcnn
print(mtcnn.__version__)

The first step is to load an image as a NumPy array, which we can achieve using the PIL library and the open() function. We will also convert the image to RGB, just in case the image has an alpha channel or is black and white.

In [6]:
# load image from file
from PIL import Image
image = Image.open('../input/5-celebrity-faces-dataset/train/ben_afflek/httpimagesfandangocomrImageRendererredesignstaticimgnoxportraitjpgpcpcpcimagesmasterrepositoryperformerimagespjpg.jpg')
# convert to RGB, if needed

image = image.convert('RGB')
# convert to array
pixels = np.asarray(image)
plt.imshow(image)

Next, we can create an MTCNN face detector class and use it to detect all faces in the loaded photograph.

In [94]:
detector = mtcnn.MTCNN()
results = detector.detect_faces(pixels)
results

Draw a rectangle arround the face using result retutn

In [8]:
x = abs(results[0]['box'][0])
y = abs(results[0]['box'][1])
xd = results[0]['box'][2]
yd = results[0]['box'][3]
x2 = x + xd
y2 = y +yd
fig, ax = plt.subplots()
rect = patches.Rectangle((x, y), xd, yd, linewidth=1, edgecolor='g', facecolor='none')
ax.add_patch(rect)
ax.imshow(image)
plt.show()

use the coordinates to extract the face.

In [9]:
face = pixels[y:y2, x:x2]
image = Image.fromarray(face)
image = image.resize((160, 160))
plt.imshow(image)

Typing all of this together, the function extract_face() will load a photograph from the loaded filename and return the extracted face. It assumes that the photo contains one face and will return the first face detected.

In [10]:
def extract_faces(filename):
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    detector = mtcnn.MTCNN()
    results = detector.detect_faces(pixels)
    x = abs(results[0]['box'][0])
    y = abs(results[0]['box'][1])
    xd = results[0]['box'][2]
    yd = results[0]['box'][3]
    x2 = x + xd
    y2 = y + yd
    face = pixels[y:y2, x:x2]
    image = Image.fromarray(face)
    image = image.resize((160, 160))
    face_array = np.asarray(image)
    return (face_array)

# 2. Face Classification System

In this section, we will develop a face detection system to predict the identity of a given face.

The model will be trained and tested using the ‘5 Celebrity Faces Dataset‘ that contains many photographs of five different celebrities.

In [11]:
from os import listdir
from os.path import isdir

i = 1
folder = '../input/5-celebrity-faces-dataset/train/ben_afflek/'
for filename in listdir(folder):
    path = folder + filename
    face = extract_faces(path)
    print(i, face.shape)
    plt.subplot(7, 7, i)
    plt.axis('off')
    plt.imshow(face)
    i += 1
    

Tansform the previous code to a function to load_faces() that take a folder name and return all faces

In [12]:
def load_faces(folder):
    faces = list()
    for filename in listdir(folder):
        path = folder + filename
        face = extract_faces(path)
        faces.append(face)
    return (faces)

The load_dataset function below will return a list of images and a label name that we eill take it from folder name. We will implement this function in the train and the  val folder.

In [13]:
def load_data(folder):
    x, y = list(), list()
    for subdir in listdir(folder):
        path = folder + subdir +'/'
        if not isdir(path):
            continue
        faces = load_faces(path)
        labels = [subdir] * len(faces)
        x.extend(faces)
        y.extend(labels)
    return np.asarray(x), np.asarray(y)

Load train Data and test Data (generate a face embeded that the model will use to classificy the new data)

In [14]:
x_train, y_train = load_data('../input/5-celebrity-faces-dataset/train/')
x_test, y_test = load_data('../input/5-celebrity-faces-dataset/val/')
x_train.shape, y_train.shape, x_test.shape, y_test.shape

We will save the data in a zip file to use it next time, without neeed to build it again,

In [15]:
np.savez_compressed('5-celebrity-faces.npz', x_train, y_train, x_test, y_test)

Load the Data from the Zip file

In [72]:
data = np.load('5-celebrity-faces.npz')
x_train, y_train, x_test, y_test = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (x_train.shape[0], x_test.shape[0]))


Next, we can load our FaceNet model ready for converting faces into face embeddings.

In [98]:
# load the model
model = load_model('../input/facenet-keras/facenet_keras.h5')
# summarize input and output shape
print(model.inputs)
print(model.outputs)

To predict an embedding, first the pixel values of the image need to be suitably prepared to meet the expectations of the FaceNet model. This specific implementation of the FaceNet model expects that the pixel values are standardized, and we have to expand the dimension so that the face array is one simple, all that wille be in get_embedding() function

In [41]:
def get_embidding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = np.expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    prediction = model.predict(samples)
    return prediction[0]

Now we eiil convert each face in our data to faceembidding using the above function

In [99]:
#transform train_data_x
x_train_embd = list()
for face_pixel in x_train:
    embd = get_embidding(model, face_pixel)
    x_train_embd.append(embd)
x_train_embd = np.asarray(x_train_embd)

#transform test_data_x
x_test_embd = list()
for face_pixel in x_test:
    embd = get_embidding(model, face_pixel)
    x_test_embd.append(embd)
x_test_embd = np.asarray(x_test_embd)
x_test_embd.shape, x_train_embd.shape

## **Perform Face Classification**

First, it is a good practice to normalize the face embedding vectors. It is a good practice because the vectors are often compared to each other using a distance metric.

In [75]:
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
# normalize input vectors
in_encoder = Normalizer(norm='l2')
x_train_embd = in_encoder.transform(x_train_embd)
x_test_embd = in_encoder.transform(x_test_embd)
x_test_embd, x_train_embd

Next, the string target variables for each celebrity name need to be converted to integers.

In [76]:
# label encode targets
out_encoder = preprocessing.LabelEncoder()
out_encoder.fit(y_train)
y_train = out_encoder.transform(y_train)
y_test = out_encoder.transform(y_test)
y_test

To build a classification MODEL we can use SVC (SUPPORT VECTOR MACHINE) algorithm that is more acuraccy, then we will train our model

In [80]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 

model = SVC(kernel='linear', probability=True)

#train the project
model.fit(x_test_embd, y_test)

y_predect_train = model.predict(x_train_embd)
y_predect_test = model.predict(x_test_embd)
#calcule the ACURRACY
score_train = accuracy_score(y_train, y_predect_train)
score_test = accuracy_score(y_test, y_predect_test)
print("accuracy for tarin data : {}\naccuracy for test data  : {}".format(score_train * 100, score_test  * 100))

In [81]:
def load_data_2(folder):
    x, y = list(), list()
    for filename in listdir(folder):
        path = folder + filename
        face = extract_faces(path)
        labels = [filename]
        x.append(face)
        y.extend(labels)
    return np.asarray(x), np.asarray(y)

In [93]:
folder = '../input/newimagefortest/'
new_faces, new_names = load_data_2(folder)
model_embedd = load_model('../input/facenet-keras/facenet_keras.h5')
i = 0
for face in new_faces:
    face_embd = get_embidding(model_embedd, np.asarray(face))
    prob = model.predict_proba([face_embd])
    out = model.predict([face_embd])
    out_name = out_encoder.inverse_transform(out)
    print("predict name :  ", out_name[0], "  with prob :  ", prob[0, out] * 100 , "   correct name is :       ", new_names[i][:-5])
    i += 1

We will test our model using new image one image for each label